In [ ]:
!pip install --upgrade --quiet gspread
!pip install nltk

In [ ]:
#Import necessary libraries

import pandas as pd
import numpy as np
import urllib.request
import json
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.stats import linregress
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
import re
import requests
import nltk
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sentVader
nltk.download('vader_lexicon')
nltk.download('punkt')

In [ ]:
#Helper Functions and data

def get_trump_tweets():
  # This function grabs the Trump tweets from the CTASS GitHub
  return pd.read_json('https://raw.githubusercontent.com/ctass/fall-workshop-2019/master/trump_tweets.json')

party_name_dict = pd.read_json('https://raw.githubusercontent.com/ctass/fall-workshop-2019/master/moc_handle_dict.json')

def get_cong_tweet(date):
  #This function takes a date such as '2018-12-25' as a parameter and outputs
  #the tweets of all of the members of congress' tweets on that day. We combine
  #this data with a simple dictionary that contains MoCs' twitter handles,
  #names, and party affiliation.
  url = 'https://raw.githubusercontent.com/alexlitel/congresstweets/master/data/{}.json'.format(date)
  tweets = pd.read_json(url)
  global party_name_dict
  return tweets.merge(party_name_dict, on='screen_name')

def get_mfd():
  #This function grabs a version of the moral foundations dictionary from the
  #CTASS GitHub which can be used with the dict_vec function below
  return requests.get('https://raw.githubusercontent.com/ctass/fall-workshop-2019/master/mfd.dict').json()

def get_news(source, date):
    #This function grabs the front page of MSNBC or Fox News on a particular day
    #returns it as a single string. These were sourced from the Internet Archive 
    #and stored on the CTASS Github. The function takes a source (either 'msnbc'
    #or 'fox') and dates of the form YYYY-MM-DD. News has been 
    #archived from '2019-10-21' to '2019-11-21' (inclusive), except that for a
    #couple of days we had to grab a post near midnight the next day.
    base = 'https://raw.githubusercontent.com/ctass/fall-workshop-2019/master/news/'
    url = base + source + '-' + date.replace('-', '') + '.txt'
    data = urllib.request.urlopen(url)
    return data.read().decode('utf-8').replace('\n', ' ').lower()
    
def regexify(s):
  #Helper function for dictify below
  if s[-1] == "*":
    return r'(?<!\w){}\w*?[\s.]'.format(s[:-1])
  return r'(?<!\w){}[\s.]'.format(s)

def dictify(d):
  #Takes in a dictionary where keys are categories and the values are lists of
  #words that indicate each category. Outputs a dictionary that can be used by
  #dict_vec below. Helpful if you are not comfortable with regular expressions,
  #but is limited by design. There is one key functionality built in: if a word
  #ends in a "*", then the generated regular expression will look for words with
  #any additional letters at the end of it (e.g. "eat*" captures "eat", 
  #"eating", "eats", etc.). The same does not hold for "*" at the beginning of a 
  #word.
  return {k: [regexify(w) for w in d[k]] for k in d}

def dict_vec(s, d):
  #Takes in a string and a dictionary which is structured {C1: [W11, W21, ...],
  #                                                        C2: [W12, W22, ...]}
  #Where Cx is the name of a category and Wxy is a regular expression which
  #indicates a string 'more belongs' in category x and outputs a dictionary
  #structured {C1: S1, C2: S2, ...} where Cx is the name of a category and
  #Sx is how much the passed string 'belongs' in that category.
  results = {k: 0 for k in d.keys()}
  results['_total_'] = 0
  s = s.lower().translate(str.maketrans('', '', string.punctuation))
  for k in d.keys():
    for t in d[k]:
      count = len(re.findall(t,s.lower()+' '))
      results[k] += count
      results['_total_'] += count
  results['_len_'] = len(nltk.tokenize.word_tokenize(s))
  return results


#**Accessing Three Available Corpora**

For the purposes of this workshop we've made starter code that allows you to easily access three different text corpora: (1) DJT's tweets, (2) Tweets of every member of the US congress, and (3) the front pages of MSNBC.com and FoxNews.com. We've attempted to make the starter code as straight-forward as possible, and below provide code that you can copy and paste for many analyses. **DISCLAIMER:** all of the corpora come from second-hand sources. Therefore, we cannot speak to the accuracy of these data, though from some intial snooping around the data seems to check out.

##Donald Trump's Tweets since the Inauguration
We have a little helper function that allows you to download DJT's tweets from
the CTASS GitHub. It will automatically output it as a Pandas DataFrame,
which is a useful tool for doing data science in Python. See below for an
example of loading the tweets and assigning it to a variable df. We also
remove retweets (you don't have to) and drop the id_str variable

In [ ]:
df = get_trump_tweets()
df = df[df['is_retweet']==0.0].drop(['is_retweet','id_str'],axis=1)
df

##Tweets from Members of Congress (MoCs)
Next we have a function that takes in a date (e.g. '2017-11-17') and spits back
all the tweets sent out by any member of congress on that day. Importantly,
this data is maintained by the GitHub user alexlitel, so our ability to access
this data is all due to them. Thank you whoever you are! Importantly, we don't
have access to the number of favorites or retweets of these tweets. Also, we only have access to tweets starting on June 21st of 2017. The function then also matches these Twitter handles to the MoC's name and their party affiliation, which combines data maintained by triagecancer.org, opendatasoft.com, and opensecrety.org. We'll drop some columns that don't on their face appear to be helpful.

In [ ]:
get_cong_tweet('2017-11-17').drop(['id','user_id'], axis=1)

##The front pages of Foxnews.com and MSNBC.com

Lastly, we have a small corpus we've collected (with the help of the Wayback machine internet archive) of the front page of msnbc.com and foxnews.com. We've collected all of the text from the front of these websites from October 21st to November 21st, with the one caveat that for a couple of days (November 7th and November 10th) we couldn't grab the front page of MSNBC, so we instead use a grab from around midnight the next day. The function takes in the source (either 'msnbc' or 'fox') and the date (e.g. '2019-11-19' for November 11th of 2019) and returns back a string that isn't that pretty but contains all the relevant information. Since we're primarily exploring dictionary methods here this won't be too much of a problem, but you could imagine that if we wanted to use more complicated methods that leveraged the structure of language we might need to be a little more sophisticated with how we collect and clean this data. 

In [ ]:
get_news('msnbc','2019-11-19')

#**Example Analyses Using the Data**
Next, we do some basic data analysis and visualization. If you're not very comfortable doing analyses in Python, copying and pasting this code and changing a few variables here and there is a great way to get started. Regardless of your comfort level with Python, these are some fairly simple analyses that can get your creativity going to come up with your own analyses. There's plenty of different ways we could have done any of these analyses; we emphasized simplicity of code and analyses that are easy to follow.

##Basic Data Visualization and Analysis in Colab

We're going to use a library called Seaborn for visualization. There are many
other tools that are worth looking into, but Austin likes Seaborn because
it's pretty easy to use, especially if you're already familiar with matplot,
and makes reasonably pretty graphs (big step up from standard Stata graphs at least). This command just sets the style of our graphs. Austin likes this particular style, but you can also try 'darkgrid', 'dark', 'white', or 'ticks'

In [ ]:
sb.set_style('whitegrid')

Here we'll produce a regression plot of the relationship between the
number of retweets a tweet has and the number of favorites it has.
We'll then do a regression using scipy to get a p-value (though we can see from
the bootstrapped confidence intervals in the plot that this is relationship
is highly significant). One thing that would definitely make this plot more
legible is setting bounds on the x and y axis to exclude outliers.

In [ ]:
plt.scatter(df['retweet_count'],df['favorite_count'], alpha=0.25)
#NOTE: "alpha" is a parameter that controls how transparent the scatterplot
#points are in the scatterplot
sb.regplot(df['retweet_count'],df['favorite_count'],scatter=False)
plt.xlabel('# of Retweets')
plt.ylabel('# of Favorites')
plt.title("The relationship between RTs and favorites for DJT's tweets.")
plt.show()
print('Does the number of favorites increase with the number of retweets?')
print(linregress(df['retweet_count'],df['favorite_count']))

Here we'll look at the mean number of retweets and favorites for tweets
from different sources. You can get a sense from the confidence intervals
which tweet sources are more common in the data. I would consider dropping
the Twitter Web Client category from this analysis.

In [ ]:
mean_retweets = df.groupby('source')['retweet_count'].mean()
std_retweets = df.groupby('source')['retweet_count'].std()

mean_favorites = df.groupby('source')['favorite_count'].mean()
std_favorites = df.groupby('source')['favorite_count'].std()

source_rt_name, source_rt_code = np.unique(mean_retweets.index, 
                                           return_inverse=True)
source_fv_name, source_fv_code = np.unique(mean_favorites.index, 
                                           return_inverse=True)

plt.errorbar(source_rt_code, mean_retweets, xerr=0.2, yerr=2*std_retweets, 
             linestyle='', label='Retweets')
plt.errorbar(source_rt_code, mean_favorites, xerr=0.2, yerr=2*std_retweets, 
             linestyle='', label='Favorites')

plt.xticks(source_rt_code, source_rt_name, rotation=30, ha='right')
plt.legend()
plt.title("RTs and favorites by source of DJT's tweet.")
plt.xlabel('Source of tweet')
plt.ylabel('Count')
plt.show()

Here we're going to test whether Trump's tweets received more favorites and
retweets on average in 2018 when compared to tweets in 2017. I excluded the
2019 tweets because it makes the graphs look busy, but the command to plot them
is simply commented out. Note that like many of our analyses here, our outcome is technically a count variable and should be analyzed by something that takes such a fact into account such as a negative binomial model, but also note from the histogram that it's incredibly unlikely that such a model would not return a significant association.

In [ ]:
rt = {2017: [],
      2018: [],
      2019: []}

fv = {2017: [],
      2018: [],
      2019: []}

for i, row in df.iterrows():
    yr = row['created_at'].year
    if row['retweet_count'] < 100000:
      rt[yr].append(row['retweet_count']) 
    if row['favorite_count'] < 300000:
      fv[yr].append(row['favorite_count'])

sb.distplot(rt[2017], label='2017')
sb.distplot(rt[2018], label='2018')
#sb.distplot(rt[2019], label='2019')

plt.legend()
plt.xlabel('# of favorites')
plt.ylabel('Frequency')
plt.title('Number of retweets by year.')

plt.show()

print("Is there a mean difference in the number of retweets for the years 2017 and 2018?")
print(ttest_ind(rt[2017],rt[2018],equal_var=False))

sb.distplot(fv[2017], label='2017')
sb.distplot(fv[2018], label='2018')
#sb.distplot(fv[2019], label='2019')

plt.legend()
plt.xlabel('# of favorites')
plt.ylabel('Frequency')
plt.title('Number of favorites by year.')

plt.show()

print("Is there a mean difference in the number of favorites for the years 2017 and 2018?")
print(ttest_ind(fv[2017],fv[2018],equal_var=False))

##Sentiment Analysis

This is a quick demo of how to use Vader, which is a sentiment analysis tool
developed by the NLTK group here at Stanford. It's basically a sentiment
lexicon, but uses some smart heuristics to do a slightly better job.
You can get degree to which a string is positive, neutral, or negative, or you
get a compound or summary score

In [ ]:
sent = sentVader()

print(sent.polarity_scores('Everything about this is awesome. Except for the bagels, which are amazing'))
print(sent.polarity_scores('Everything about this is terrible. Except for the bagels, which are amazing'))
print(sent.polarity_scores('Everything about this is terrible. Except for the bagels, which are still not amazing'))

What can we do with sentiment analysis? Let's see whether Democrats and
Republicans tweeted differently sentimented language this Wednesday, November 20th. 

In [ ]:
df = get_cong_tweet('2019-11-20')

Then, we'll, create two lists; corresponding to the sentiments of tweets
tweeted by Democrats and Republicans, respectively

In [ ]:
dems = []
pubs = []
sent = sentVader()

for i, row in df.iterrows():
  s = sent.polarity_scores(row['text'])['compound']
  if row['party'] == 'Democrat':
    dems.append(s)
  elif row['party'] == 'Republican':
    pubs.append(s)

Lastly, we'll produce a graph showing the histograms of sentiments by the two
parties and do a simple t-test to asses whether there is a statistically
significant difference

In [ ]:
sb.distplot(dems, label='Democrats', color='skyblue')
sb.distplot(pubs, label='Republicans', color='crimson')

plt.legend()
plt.xlabel('Sentiment of tweet')
plt.ylabel('Frequency')
plt.title('Sentiment of tweets this Wednesday.')

print(ttest_ind(dems, pubs, equal_var=False))

Just as a cautionary note: often with computational methods you'll get crazy distributions like this, which don't necessarily meet the assumptions of different statistical tests such as a t-test, making your p-values invalid. It's nice to have a good understanding of statistics so that you know when you're breaking these rules, and how you can do analyses that don't make such assumptions. For instance, here (and in many other analyses we do here) you could do a Fischer randomization test, and specifically here the test statistic could be the mean of the sentiment score. The Fischer randomization test is model-free and doesn't make any assumption about the marginal distributions of the outcome variable.

##(Other) Dictionary Methods

Now we've got a series of functions to help you do other dictionary-
based analyses. First up is this get_mfd function, which downloads a Python-
friendly version of the Moral Foundations Dictionary from the CTASS GitHub.
The dictionary isn't made up words per se, but instead regular expressions,
which allow for efficient searching of resonably complicated patterns in text.
There are other ways we could have coded this, but we preferred this for its
flexibility and its scalability (in terms of the length of analyzed texts).

In [ ]:
mfd = get_mfd()
mfd

The next function takes in any such dictionary (you can make your own! See 
below) and outputs a dictionary indicating a string's score on each of the
dictionary's categories. It also creates a \_total\_ category (the sum of all the other categories) as well as a \_len\_ category (the number of words in the string), in case you want to normalize the scores of the other categories.

In [ ]:
dict_vec('This is an unjust test string. It is a very pure test string.',mfd)

Here's a demo of defining your own dictionary for analysis. Though you can
certainly do this with one line of code, we'll break into steps so it's easier
to follow. First, define lists of words that relate to the different categories
in your dictionary. You can put a "*" at the end of a word to capture any word
that starts with those letters. For instance, "eat*" will capture "eat",
"eating", and "eats", where "eat" will only capture "eat".

In [ ]:
politic_words = ['politi*', 'congress*', 'president*', 'democrat', 'republican']
food_words = ['tuna', 'pineapple', 'burger', 'din*', 'eat*']

Then define a dictionary in which the key is a name for each category in the
dictionary and the value are the appropriate lists you defined before


In [ ]:
new_dict = {'politics': politic_words, 
            'food': food_words}

Then lastly pass that dictionary into our dictify function, which just turns
your words into regular expressions for you!

In [ ]:
new_dict = dictify(new_dict)

Then you can use your dictionary just like how we used the MF dictionary!

In [ ]:
dict_vec('These congressional hearings are eating him alive', new_dict)

So what can we do with these dictionary methods? Moral foundations theory
posits that conservatives care more about a subset of moral dimensions (namely
purity, authority, and ingroup) than liberals. Something that would be
consistent with such a theory would be if FoxNews.com (which has a more
conservative viewership) used more language that appealed to such moral
foundations than MSNBC.com (which has a more liberal viewership). So, we'll
simply compare the text on the front pages of these two websites and see the
degree to which these two websites use words from those sections of the moral
foundations dictionary more.
We'll iterate over each day of data that we have. For each day, we'll load the
text from the front pages of the two websites. For each website, we'll get the
count of words in the vice and virtue categories associatied with ingroup,
authority, and purity, and add that number normalized by the number of words on the front page to lists we maintain.

In [ ]:
msnbc_pur = []
msnbc_auth = []
msnbc_ingrp = []

fox_pur = []
fox_auth = []
fox_ingrp = []

for date in ['2019-10-21','2019-10-22','2019-10-23','2019-10-24','2019-10-25',
             '2019-10-26','2019-10-27','2019-10-28','2019-10-29','2019-10-30',
             '2019-10-31',
             '2019-11-01','2019-11-02','2019-11-03','2019-11-04','2019-11-05',
             '2019-11-06','2019-11-07','2019-11-08','2019-11-09','2019-11-10',
             '2019-11-11','2019-11-12','2019-11-13','2019-11-14','2019-11-15',
             '2019-11-16','2019-11-17','2019-11-18','2019-11-19','2019-11-20',
             '2019-11-21']:

  msnbc = get_news('msnbc',date)
  msnbc_vec = dict_vec(msnbc, mfd)
  m_len = msnbc_vec['_len_']

  fox = get_news('fox',date)
  fox_vec = dict_vec(fox, mfd)
  f_len = fox_vec['_len_']

  msnbc_pur.append((msnbc_vec['PurityVirtue'] + msnbc_vec['PurityVice'])/m_len)
  msnbc_auth.append((msnbc_vec['AuthorityVirtue'] + msnbc_vec['AuthorityVice'])/m_len)
  msnbc_ingrp.append((msnbc_vec['IngroupVirtue'] + msnbc_vec['IngroupVice'])/m_len)

  fox_pur.append((fox_vec['PurityVirtue'] + fox_vec['PurityVice'])/f_len)
  fox_auth.append((fox_vec['AuthorityVirtue'] + fox_vec['AuthorityVice'])/f_len)
  fox_ingrp.append((fox_vec['IngroupVirtue'] + fox_vec['IngroupVice'])/f_len)

Now, we're going to make a weird assumption and use a paired sample t-test, with the idea that we're analyzing whether the objective day of events is describesd differently by
Fox and MSNBC, which we're conceptualizing as the condition. We're further going
to treat what is obviously a fractional variable as a continuous variable, just
for sake of implementation and such. Definitely different (slash better) ways 
one could do this analysis!

In [ ]:
sb.distplot(msnbc_pur, label='MSNBC', color='skyblue')
sb.distplot(fox_pur, label='Fox', color='crimson')
plt.legend()
plt.xlabel('Count of purity words')
plt.ylabel('Frequency')
plt.title('Do Fox and MSNBC use different amount of purity language?')
plt.show()
print(ttest_rel(msnbc_pur, fox_pur))

sb.distplot(msnbc_auth, label='MSNBC', color='skyblue')
sb.distplot(fox_auth, label='Fox', color='crimson')
plt.legend()
plt.xlabel('Count of authority words')
plt.ylabel('Frequency')
plt.title('Do Fox and MSNBC use different amount of authority language?')
plt.show()
print(ttest_rel(msnbc_auth, fox_auth))

sb.distplot(msnbc_ingrp, label='MSNBC', color='skyblue')
sb.distplot(fox_ingrp, label='Fox', color='crimson')
plt.legend()
plt.xlabel('Count of ingroup words')
plt.ylabel('Frequency')
plt.title('Do Fox and MSNBC use different amount of ingroup language?')
plt.show()
print(ttest_rel(msnbc_ingrp, fox_ingrp))

#**Your turn!**